# File to analyze graphics of default simulation
------------------------------------------------

## Imports to collect data and plot graphics

In [ ]:
from components.graphic_generator import Graphic_Gen
from components.data_manager import Data_Manager
import components.network_data as nwd
from components.utils.enums import Colors
import numpy as np
import pandas as pd

## Constants

In [ ]:
PATH_TO_CSV: str = "data/default_simulation/default_simulation.csv"
NUMBER_OF_GRAPHIC_POINTS: int = 7
BLACK_HOLES_NUMBER: list = [1, 3, 5]
INTENSITIES: list = [intensity*0.1 for intensity in range(1, 8)]
STANDARD_DEVIATION: bool = False
COLORS_LIST: list[Colors] = [Colors.RED, Colors.ORANGE, Colors.YELLOW]

## Load csv

In [ ]:
dm: Data_Manager = Data_Manager()
loaded: bool = dm.load_csv(PATH_TO_CSV)
if not loaded:
    print("File don't loaded")

## Convert csv to pandas DataFrame

In [ ]:
dframe: pd.DataFrame = dm.csv_to_dataframe()

## Manipulate data to collect columns from default network (without black holes)

In [ ]:
default_consummed_eprs: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.CONSUMED_EPRS])
default_success_tax: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.TOTAL_REQUEST_SUCCESS])
default_avg_fidelity: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.TOTAL_ROUTE_FIDELITY])

default_number_requests: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.REQUESTS])
default_number_total_requests_success: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.TOTAL_REQUEST_SUCCESS])

points_number: int = len(INTENSITIES)
default_axle_consummed_eprs: np.ndarray = np.tile((default_consummed_eprs).to_numpy(), (points_number, 1))
default_axle_success_tax: np.ndarray = np.tile(((default_success_tax / default_number_requests) * 100).to_numpy(), (points_number, 1))
default_axle_avg_fidelity: np.ndarray = np.tile((default_avg_fidelity / default_number_total_requests_success).to_numpy(), (points_number, 1))

## Function to collect attacked data from each number of black holes

In [ ]:
def collect_axis_list(attack_column: str, divisor_column: str | None, targets: int, multiply_factor: int = 1) -> np.ndarray:
    attacked_axis_list: list[np.ndarray] = []
    for bh_number in BLACK_HOLES_NUMBER:
        attacked_axis: list[np.ndarray] = []
        for i in range(1, NUMBER_OF_GRAPHIC_POINTS+1):  
            mask: pd.Series = (
                (dframe[nwd.INTENSITY] == f"i: {(i*0.1):.1f}") &
                (dframe[nwd.TARGETS_PER_BLACK_HOLE] == targets) &
                (dframe[nwd.NUMBER_OF_BLACK_HOLES] == bh_number)
            )
            attack_serie: pd.Series = (dframe.loc[mask, attack_column])

            if divisor_column is not None:
                divisor_serie: pd.Series = (dframe.loc[mask, divisor_column])
                data: np.ndarray = ((attack_serie / divisor_serie) * multiply_factor).to_numpy()
            else:
                data: np.ndarray = (attack_serie * multiply_factor).to_numpy()

            attacked_axis.append(data)
        attacked_axis_list.append(np.array(attacked_axis))
    
    return np.array(attacked_axis_list)

## Use the function to collect axis list

In [ ]:
attacked_axis_consummed_eprs_no_targets: np.ndarray = collect_axis_list(attack_column=nwd.CONSUMED_EPRS, divisor_column=None, targets=0)
attacked_axis_consummed_eprs_with_targets: np.ndarray = collect_axis_list(attack_column=nwd.CONSUMED_EPRS, divisor_column=None, targets=1)

attacked_axis_success_tax_no_targets: np.ndarray = collect_axis_list(attack_column=nwd.TOTAL_REQUEST_SUCCESS, divisor_column=nwd.REQUESTS, targets=0, multiply_factor=100)
attacked_axis_success_tax_with_targets: np.ndarray = collect_axis_list(attack_column=nwd.TOTAL_REQUEST_SUCCESS, divisor_column=nwd.REQUESTS, targets=1, multiply_factor=100)

attacked_axis_avg_fidelity_no_targets: np.ndarray = collect_axis_list(attack_column=nwd.TOTAL_ROUTE_FIDELITY, divisor_column=nwd.TOTAL_REQUEST_SUCCESS, targets=0)
attacked_axis_avg_fidelity_with_targets: np.ndarray = collect_axis_list(attack_column=nwd.TOTAL_ROUTE_FIDELITY, divisor_column=nwd.TOTAL_REQUEST_SUCCESS, targets=1)

--------------
## Success Tax
--------------

### Success tax without target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_success_tax_no_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_success_tax, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_success_tax, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Success Tax (target off)", x_label="Intensity", y_label="Diff Success Tax", grid=True)

### Success tax with target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_success_tax_with_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_success_tax, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_success_tax, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Success Tax (target on)", x_label="Intensity", y_label="Diff Success Tax", grid=True)

-----------------
## Consummed EPRs
-----------------

### Consumed EPRs without targets

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = attacked_axis_consummed_eprs_no_targets[i]
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_consummed_eprs, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_consummed_eprs, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Consummed EPRs (target off)", x_label="Intensity", y_label="Diff Consummed Eprs", grid=True)

### Consumed EPRs with target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_consummed_eprs_with_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_consummed_eprs, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_consummed_eprs, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Consummed EPRs (target on)", x_label="Intensity", y_label="Diff Consummed Eprs", grid=True)

-----------------
## Route Fidelity
-----------------

### Average Route Fidelity without target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_avg_fidelity_no_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Avg Fidelity (target off)", x_label="Intensity", y_label="Diff Avg Fidelity", grid=True)

### Average Route Fidelity with target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_avg_fidelity_with_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=STANDARD_DEVIATION, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Avg Fidelity (target on)", x_label="Intensity", y_label="Diff Avg Fidelity", grid=True)

## Correlation Graphic

In [ ]:
df: pd.DataFrame = dframe.drop(
    columns=[
        nwd.INTENSITY,
        nwd.NORMAL_NODE_SWAP_PROB,
        nwd.REQUESTS,
        nwd.TOTAL_NO_PATHS,
        nwd.NUMBER_OF_NODES,
        nwd.PARAMETER,
        nwd.NUMBER_OF_NODES,
        nwd.TOPOLOGY 
    ]
) # remove constants columns and string columns

Graphic_Gen.plot_heatmap(df.corr(), title="Heatmap of Correlation in Topology Simulations")